In [ ]:
# ==========================================
# 1. CÀI ĐẶT THƯ VIỆN
# ==========================================
%pip install -q transformers datasets accelerate evaluate sentencepiece vncorenlp

import json
import re
import torch
import collections
import numpy as np
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForQuestionAnswering,
    TrainingArguments,
    Trainer,
    default_data_collator
)
from vncorenlp import VnCoreNLP

In [ ]:
# ==========================================
# 2. CẤU HÌNH VNCORENLP
# ==========================================
# Đường dẫn file jar bạn đã upload
VNCORENLP_JAR = '/content/vncorenlp/VnCoreNLP-1.1.1.jar'

try:
    # Khởi tạo VnCoreNLP (chỉ dùng annotator 'wseg' để tách từ)
    rdrsegmenter = VnCoreNLP(VNCORENLP_JAR, annotators="wseg", max_heap_size='-Xmx500m')
    print("✅ Đã khởi tạo thành công VnCoreNLP!")
except Exception as e:
    print("❌ Lỗi khởi tạo VnCoreNLP. Kiểm tra lại đường dẫn file .jar")
    print(e)
    # Tạo dummy function nếu không load được để code không crash
    class DummySegmenter:
        def tokenize(self, text): return [[text]]
    rdrsegmenter = DummySegmenter()

def segment_text(text):
    """Hàm tách từ: Input 'Hà Nội' -> Output 'Hà_Nội'"""
    if not text: return ""
    try:
        sentences = rdrsegmenter.tokenize(text)
        # Nối các từ lại bằng khoảng trắng (VnCoreNLP dùng _ nối từ ghép)
        return " ".join([" ".join(s) for s in sentences])
    except:
        return text

In [ ]:
# ==========================================
# 3. HÀM LOAD & SỬA LỖI JSON (SIÊU MẠNH - CẢI TIẾN)
# ==========================================
def load_json_robust(file_path):
    """
    Load JSON với xử lý các lỗi escape không hợp lệ
    Đặc biệt xử lý các ký tự LaTeX như \displaystyle, \in, etc.
    """
    print(f"📂 Đang đọc file: {file_path} ...")
    
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    # --- BƯỚC SỬA LỖI REGEX (CẢI TIẾN) ---
    
    # 1. Sửa lỗi \u không hợp lệ (ví dụ: đường dẫn C:\user -> C:\\user)
    # Chỉ sửa \u không theo sau bởi 4 hex digits
    content = re.sub(r'\\u(?![0-9a-fA-F]{4})', r'\\\\u', content)
    
    # 2. Sửa lỗi escape không hợp lệ trong chuỗi JSON
    # JSON chỉ cho phép các escape: ", \, /, b, f, n, r, t, u
    # Tất cả các \ khác cần được escape thành \\
    
    # Strategy: Tìm tất cả các \ trong chuỗi (không phải trong escape sequence hợp lệ)
    # và escape chúng
    
    # Trước tiên, bảo vệ các escape sequence hợp lệ bằng placeholder
    valid_escapes = {
        r'\\"': '__QUOTE_ESCAPE__',
        r'\\\\': '__BACKSLASH_ESCAPE__',
        r'\\/': '__SLASH_ESCAPE__',
        r'\\b': '__BS_ESCAPE__',
        r'\\f': '__FF_ESCAPE__',
        r'\\n': '__LF_ESCAPE__',
        r'\\r': '__CR_ESCAPE__',
        r'\\t': '__TAB_ESCAPE__',
        r'\\u[0-9a-fA-F]{4}': '__UNICODE_ESCAPE__',  # \uXXXX
    }
    
    # Bảo vệ các escape hợp lệ
    protected_content = content
    for pattern, placeholder in valid_escapes.items():
        if pattern == r'\\u[0-9a-fA-F]{4}':
            # Xử lý đặc biệt cho unicode escape
            protected_content = re.sub(r'\\u([0-9a-fA-F]{4})', r'__UNICODE_ESCAPE__\1', protected_content)
        else:
            protected_content = protected_content.replace(pattern, placeholder)
    
    # Bây giờ escape tất cả các \ còn lại (không phải là escape hợp lệ)
    # Tìm các \ đứng một mình hoặc theo sau bởi ký tự không hợp lệ
    protected_content = re.sub(r'\\(?![\\"/bfnrtu])', r'\\\\', protected_content)
    
    # Khôi phục các escape hợp lệ
    for pattern, placeholder in valid_escapes.items():
        if pattern == r'\\u[0-9a-fA-F]{4}':
            protected_content = re.sub(r'__UNICODE_ESCAPE__([0-9a-fA-F]{4})', r'\\u\1', protected_content)
        else:
            protected_content = protected_content.replace(placeholder, pattern)
    
    # 3. Xử lý trường hợp đặc biệt: các ký tự LaTeX thường gặp
    # Ví dụ: \displaystyle, \in, \sum, \int, etc.
    # Những cái này đã được xử lý ở bước 2, nhưng có thể cần cleanup thêm
    
    # 4. Cleanup: Xử lý các dấu \\\\ chồng chéo (nếu có)
    # Giảm \\\\ thành \\ (nhưng giữ \\\\ nếu là escape hợp lệ của \)
    protected_content = re.sub(r'\\\\{3,}', r'\\\\', protected_content)
    
    try:
        data = json.loads(protected_content)
        print("✅ Parse JSON thành công!")
        return data
    except json.JSONDecodeError as e:
        print(f"❌ Vẫn lỗi JSON tại vị trí {e.pos}: {e.msg}")
        start = max(0, e.pos - 200)
        end = min(len(protected_content), e.pos + 200)
        print(f"Context lỗi: ...{protected_content[start:end]}...")
        
        # Thử cách khác: xử lý từng dòng
        print("\n🔄 Thử phương pháp xử lý từng dòng...")
        try:
            lines = protected_content.split('\n')
            fixed_lines = []
            for i, line in enumerate(lines):
                # Escape các \ không hợp lệ trong mỗi dòng
                fixed_line = re.sub(r'\\(?![\\"/bfnrtu])', r'\\\\', line)
                fixed_lines.append(fixed_line)
            fixed_content = '\n'.join(fixed_lines)
            data = json.loads(fixed_content)
            print("✅ Parse JSON thành công bằng phương pháp xử lý từng dòng!")
            return data
        except Exception as e2:
            print(f"❌ Vẫn lỗi: {e2}")
            raise e

In [ ]:
# ==========================================
# 4. TIỀN XỬ LÝ DỮ LIỆU: SEGMENTATION & REMAP
# ==========================================

def preprocess_dataset_with_segmentation(raw_data_list):
    """
    Chạy tách từ cho toàn bộ dataset và tính toán lại answer_start
    """
    processed_data = []
    skipped_count = 0
    
    print("⏳ Đang chạy VnCoreNLP tách từ và map lại vị trí (có thể mất vài phút)...")
    
    for idx, item in enumerate(raw_data_list):
        if idx % 1000 == 0:
            print(f"  Đang xử lý: {idx}/{len(raw_data_list)}...")
        
        context_raw = item['context']
        question_raw = item['question']
        
        # 1. Tách từ Context và Question
        context_seg = segment_text(context_raw)
        question_seg = segment_text(question_raw)
        
        # 2. Xử lý Answer
        # Vì tách từ làm thay đổi độ dài chuỗi, ta phải tìm lại vị trí câu trả lời trong chuỗi mới
        answers_new = {"text": [], "answer_start": []}
        
        if "answers" in item and item["answers"]["text"]:
            text_raw = item["answers"]["text"][0]
            
            # Tách từ câu trả lời
            text_seg = segment_text(text_raw)
            
            # Tìm vị trí chính xác của câu trả lời đã tách từ trong context đã tách từ
            new_start = context_seg.find(text_seg)
            
            if new_start != -1:
                answers_new["text"] = [text_seg]
                answers_new["answer_start"] = [new_start]
                
                # Lưu data mới
                new_item = {
                    "id": item["id"],
                    "context": context_seg,   # Dùng context đã tách từ
                    "question": question_seg, # Dùng question đã tách từ
                    "answers": answers_new
                }
                processed_data.append(new_item)
            else:
                # Nếu không tìm thấy (do tách từ làm biến dạng quá nhiều), bỏ qua mẫu này để tránh lỗi training
                skipped_count += 1
                # Fallback: cố gắng dùng lại raw nếu cần, nhưng tốt nhất là skip để sạch data
        else:
            # Trường hợp không có câu trả lời
            new_item = {
                "id": item["id"],
                "context": context_seg,
                "question": question_seg,
                "answers": {"text": [], "answer_start": []}
            }
            processed_data.append(new_item)

    print(f"✅ Xử lý xong. Tổng: {len(raw_data_list)}. Hợp lệ: {len(processed_data)}. Bỏ qua: {skipped_count} (do lỗi map vị trí).")
    return Dataset.from_list(processed_data)

In [ ]:
# --- Main Loading Pipeline ---

FILE_PATH = '/content/train.json' # File json gốc

# Load JSON với xử lý lỗi escape
json_data = load_json_robust(FILE_PATH)
data_list = json_data['data'] if 'data' in json_data else json_data

print(f"📊 Tổng số mẫu: {len(data_list)}")

# Chạy Preprocess (Segmentation)
dataset = preprocess_dataset_with_segmentation(data_list)

# Chia Train/Test
datasets_split = dataset.train_test_split(test_size=0.1, seed=42)
print(f"📊 Train: {len(datasets_split['train'])}, Test: {len(datasets_split['test'])}")

In [ ]:
# ==========================================
# 5. TOKENIZATION & TRAINING
# ==========================================

MODEL_CHECKPOINT = "vinai/phobert-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)
MAX_LENGTH = 256
DOC_STRIDE = 128

def prepare_train_features(examples):
    examples["question"] = [q.strip() for q in examples["question"]]

    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=MAX_LENGTH,
        stride=DOC_STRIDE,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")

    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)
        sequence_ids = tokenized_examples.sequence_ids(i)
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]

        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

print("🔄 Đang tokenize dataset...")
tokenized_datasets = datasets_split.map(
    prepare_train_features,
    batched=True,
    remove_columns=datasets_split["train"].column_names,
    desc="Tokenizing"
)

print("✅ Tokenize xong!")

In [ ]:
# Setup Trainer
model = AutoModelForQuestionAnswering.from_pretrained(MODEL_CHECKPOINT)

args = TrainingArguments(
    output_dir="./phobert-viquad-vncorenlp",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    fp16=True,
    logging_steps=50,
    save_total_limit=2,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=default_data_collator,
)

print("🚀 Bắt đầu huấn luyện...")
trainer.train()

In [ ]:
# ==========================================
# 6. TEST DỰ ĐOÁN (CÓ TÁCH TỪ)
# ==========================================
def predict_answer_robust(context, question):
    # Quan trọng: Phải tách từ câu hỏi và context trước khi đưa vào model
    # vì model đã được train trên dữ liệu đã tách từ
    context_seg = segment_text(context)
    question_seg = segment_text(question)
    
    inputs = tokenizer(
        question_seg, 
        context_seg, 
        add_special_tokens=True, 
        return_tensors="pt",
        truncation="only_second",
        max_length=256
    ).to(model.device)
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    start_idx = torch.argmax(outputs.start_logits)
    end_idx = torch.argmax(outputs.end_logits) + 1
    
    pred_tokens = inputs.input_ids[0, start_idx:end_idx]
    answer = tokenizer.decode(pred_tokens, skip_special_tokens=True)
    
    # Optional: Thay thế dấu _ bằng khoảng trắng để dễ đọc
    return answer.replace("_", " ")

print("\n" + "="*30)
print("TEST DỰ ĐOÁN")
sample_test = "Hà Nội là thủ đô của nước Cộng hòa Xã hội chủ nghĩa Việt Nam."
question_test = "Thủ đô của Việt Nam là gì?"

print(f"Context: {sample_test}")
print(f"Question: {question_test}")
print(f"Prediction: {predict_answer_robust(sample_test, question_test)}")